In [1]:
##### Always import all needed libraries in the first cell
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
from sklearn.svm import SVC
#from planar_utils import plot_decision_boundary, sigmoid
from Dataset import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets
import pandas as pd
from string import punctuation
import sqlite3
import os
import sys
np.random.seed(1) # this sets the seed so that the runs are consistent

%matplotlib inline

In [14]:
notbanned = pd.read_csv("notbanned.csv", delimiter=',')

pd.set_option('display.max_columns', 5)  # Set to actually print out the full columns, change if needed
# print(notbanned.head(n=10))

threads_csv = pd.read_csv("reddit_threads.csv", delimiter=',')

pd.set_option('display.max_columns', 5)  # Set to actually print out the full columns, change if needed
# print(banned.head(n=10))

banned_threads = []
index = 0
for line in threads_csv['comments']:
    if threads_csv['banned'][index] == 'TRUE':
        banned_threads.append(str(line))
#         print(str(line))
#         print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        index += 1
    else:
        index += 1
        pass

not_banned_threads = []
notIndex = 0
for line in threads_csv['comments']:
    if threads_csv['banned'][notIndex] == 'FALSE':
        not_banned_threads.append(str(line))
#         print(str(line))
#         print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        notIndex += 1
    else:
        notIndex += 1
        pass


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
Xcomments = []
ycomments = []

for line in banned_threads:
    tempStr = ''
    for subLine in line.split('\n'):
        
    Xcomments.append(str(line))
    ycomments.append(1)
#     print(str(line))

for line in not_banned_threads:
    Xcomments.append(str(line))
    ycomments.append(0)
#     print(str(line))

vectorizer = TfidfVectorizer(min_df=0, lowercase=True)
X = vectorizer.fit_transform(Xcomments)
# print(vectorizer.get_feature_names())
# print(X.shape)

In [35]:
from sklearn.model_selection import train_test_split

comments_train, comments_test, y_train, y_test = train_test_split(Xcomments, 
                                                                  ycomments, 
                                                                  test_size=0.25, 
                                                                  random_state=400, 
                                                                  shuffle=True)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit_transform(comments_train)

X_train = vectorizer.transform(comments_train)
X_test  = vectorizer.transform(comments_test)
X_train

<40236x74991 sparse matrix of type '<class 'numpy.float64'>'
	with 1367813 stored elements in Compressed Sparse Row format>

In [37]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", round(score, 6))

Accuracy: 0.993513


In [38]:
from keras.models import Sequential
from keras import layers
from keras.layers import Reshape

input_dim = X_train.shape[1]  # Number of features
print(X_train.shape)
print(input_dim)

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10, input_shape=(10,),activation='softmax'))
model.add(layers.Dense(1, input_shape=(5,), activation='relu'))

(40236, 74991)
74991


In [39]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 10)                749920    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 750,041
Trainable params: 750,041
Non-trainable params: 0
_________________________________________________________________


In [40]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Train on 40236 samples, validate on 13412 samples
Epoch 1/20
 1660/40236 [>.............................] - ETA: 1:08 - loss: 0.0836 - accuracy: 0.9946

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history)